# Context API - Exercises

Practice implementing advanced context features.

In [ ]:
import sys
sys.path.insert(0, r'c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi')

from odibi.context import Context, PandasContext
import pandas as pd
from typing import Dict, Any, List

## Exercise 1: Snapshot Context

Implement a context that can take named snapshots and restore to them.

**Requirements:**
- `snapshot(name: str)` - Save current state with a name
- `restore(name: str)` - Restore to named snapshot
- `list_snapshots() -> List[str]` - List available snapshots
- Raise `KeyError` if snapshot name doesn't exist

In [ ]:
class SnapshotContext(PandasContext):
    """Context with named snapshot capability."""
    
    def __init__(self):
        super().__init__()
        # TODO: Add snapshot storage
    
    def snapshot(self, name: str) -> None:
        """Save current state with name."""
        # TODO: Implement
        pass
    
    def restore(self, name: str) -> None:
        """Restore to named snapshot."""
        # TODO: Implement
        pass
    
    def list_snapshots(self) -> List[str]:
        """List available snapshots."""
        # TODO: Implement
        pass

In [ ]:
# Test your implementation
ctx = SnapshotContext()

# Initial state
ctx.register('a', pd.DataFrame({'x': [1, 2]}))
ctx.snapshot('initial')

# Modified state
ctx.register('b', pd.DataFrame({'y': [3, 4]}))
ctx.snapshot('with_b')

# More modifications
ctx.register('c', pd.DataFrame({'z': [5, 6]}))

print("Current:", ctx.list_names())  # ['a', 'b', 'c']
print("Snapshots:", ctx.list_snapshots())  # ['initial', 'with_b']

# Restore to 'initial'
ctx.restore('initial')
print("After restore:", ctx.list_names())  # ['a']

## Exercise 2: Serializable Context

Add save/load functionality using pickle.

**Requirements:**
- `save(path: str)` - Serialize context to file
- `load(path: str)` - Load context from file
- Handle file I/O errors gracefully

In [ ]:
import pickle
from pathlib import Path

class SerializableContext(PandasContext):
    """Context that can be saved/loaded from disk."""
    
    def save(self, path: str) -> None:
        """Save context to file.
        
        Args:
            path: File path to save to
        """
        # TODO: Implement
        pass
    
    def load(self, path: str) -> None:
        """Load context from file.
        
        Args:
            path: File path to load from
            
        Raises:
            FileNotFoundError: If file doesn't exist
        """
        # TODO: Implement
        pass

In [ ]:
# Test serialization
ctx = SerializableContext()
ctx.register('data', pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]}))

# Save
ctx.save('test_context.pkl')
print("Saved:", ctx.list_names())

# Load into new context
ctx2 = SerializableContext()
ctx2.load('test_context.pkl')
print("Loaded:", ctx2.list_names())
print(ctx2.get('data'))

# Cleanup
Path('test_context.pkl').unlink()

## Exercise 3: Observable Context

Implement observer pattern to track context changes.

**Requirements:**
- `subscribe(callback)` - Register observer
- `unsubscribe(callback)` - Remove observer
- Notify observers on `register()` and `clear()`
- Callback signature: `(event: str, name: str | None) -> None`

In [ ]:
from typing import Callable, Set

class ObservableContext(PandasContext):
    """Context that notifies observers of changes."""
    
    def __init__(self):
        super().__init__()
        # TODO: Add observer storage
    
    def subscribe(self, callback: Callable[[str, str | None], None]) -> None:
        """Add observer."""
        # TODO: Implement
        pass
    
    def unsubscribe(self, callback: Callable[[str, str | None], None]) -> None:
        """Remove observer."""
        # TODO: Implement
        pass
    
    def _notify(self, event: str, name: str | None = None) -> None:
        """Notify all observers."""
        # TODO: Implement
        pass
    
    def register(self, name: str, df: pd.DataFrame) -> None:
        """Register and notify."""
        # TODO: Call super().register() then notify
        pass
    
    def clear(self) -> None:
        """Clear and notify."""
        # TODO: Call super().clear() then notify
        pass

In [ ]:
# Test observer pattern
events = []

def logger(event: str, name: str | None):
    events.append((event, name))
    print(f"📢 {event}: {name}")

ctx = ObservableContext()
ctx.subscribe(logger)

ctx.register('df1', pd.DataFrame({'x': [1]}))
ctx.register('df2', pd.DataFrame({'y': [2]}))
ctx.clear()

print("\nEvents:", events)
# Expected: [('register', 'df1'), ('register', 'df2'), ('clear', None)]

## Exercise 4: Validating Context

Add schema validation for registered DataFrames.

**Requirements:**
- `set_schema(name: str, schema: Dict[str, type])` - Define expected schema
- Validate on `register()` - raise `ValueError` if schema doesn't match
- Schema format: `{'column_name': dtype}`

In [ ]:
class ValidatingContext(PandasContext):
    """Context with schema validation."""
    
    def __init__(self):
        super().__init__()
        # TODO: Add schema storage
    
    def set_schema(self, name: str, schema: Dict[str, type]) -> None:
        """Define expected schema for a DataFrame.
        
        Args:
            name: DataFrame identifier
            schema: Dict mapping column names to dtypes
        """
        # TODO: Implement
        pass
    
    def _validate_schema(self, name: str, df: pd.DataFrame) -> None:
        """Check if DataFrame matches schema.
        
        Raises:
            ValueError: If schema doesn't match
        """
        # TODO: Implement validation logic
        # Check: columns exist, types match
        pass
    
    def register(self, name: str, df: pd.DataFrame) -> None:
        """Register with validation."""
        # TODO: Validate if schema exists, then register
        pass

In [ ]:
# Test validation
import numpy as np

ctx = ValidatingContext()

# Define schema
ctx.set_schema('customers', {
    'customer_id': np.int64,
    'name': object
})

# Valid DataFrame
valid_df = pd.DataFrame({
    'customer_id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Carol']
})
ctx.register('customers', valid_df)
print("✅ Valid DataFrame registered")

# Invalid DataFrame (missing column)
invalid_df = pd.DataFrame({'customer_id': [1, 2]})
try:
    ctx.register('customers', invalid_df)
except ValueError as e:
    print(f"❌ Validation error: {e}")

# Invalid DataFrame (wrong type)
invalid_df2 = pd.DataFrame({
    'customer_id': ['a', 'b'],  # Should be int
    'name': ['Alice', 'Bob']
})
try:
    ctx.register('customers', invalid_df2)
except ValueError as e:
    print(f"❌ Validation error: {e}")

## Bonus Exercise: Combine Features

Create a `SuperContext` that combines:
- LRU caching from lesson
- Snapshots (Exercise 1)
- Observability (Exercise 3)

Challenge: Handle interaction between features (e.g., should snapshots include observers?)

In [ ]:
# Your implementation here
class SuperContext(Context):
    """Context with multiple advanced features."""
    pass